# Worksheet 08

Name: Muniruddin Ahmed Siddiqui  
UID: U42483225

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [16]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.919475351130517, 7.52975018421269, 4.168528452196004, 5.131700930723313, 5.07773614097884, 4.056745510565915, 5.15598286007786, 3.6263679375341975, 5.929164230830251, 4.922928569622708]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [17]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.70303506123015, 8.425073600135251, 6.058001101054962, 8.16599915871396, 8.26038884862663, 8.496033616191022, 8.074364814207303, 7.642846394028064, 8.786395077531184, 9.631291726546957]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [18]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop(-1)
        data.append(p1)
    else:
        p2 = s2.pop(-1)
        data.append(p2)
print(data)

[4.922928569622708, 5.929164230830251, 3.6263679375341975, 9.631291726546957, 8.786395077531184, 5.15598286007786, 7.642846394028064, 4.056745510565915, 5.07773614097884, 8.074364814207303]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean_1 = 5, var_1 = 1, prob_1 = 0.5  
mean_2 = 8, var_2 = 1, prob_2 = 0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [19]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.922928569622708, 5.929164230830251, 3.6263679375341975, 5.15598286007786, 4.056745510565915, 5.07773614097884]
[9.631291726546957, 8.786395077531184, 7.642846394028064, 8.074364814207303]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.7948208749349615,  mean_2 = 8.533724503078378
var_1 = 0.5706104946973347,  var_2 = 0.5682928395429074


c:\Users\munir\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The mean values are close to the true values, but the variance values are quite different. The probabilites are also slightly different.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [20]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.922928569622708
probability of observing that point if it came from cluster 0 =  0.6817498831026917
probability of observing that point if it came from cluster 1 =  1.202396801937155e-09
point =  5.929164230830251
probability of observing that point if it came from cluster 0 =  0.09692124907238202
probability of observing that point if it came from cluster 1 =  1.928153018196993e-05
point =  3.6263679375341975
probability of observing that point if it came from cluster 0 =  0.08590808342103569
probability of observing that point if it came from cluster 1 =  4.5098627371948446e-17
point =  9.631291726546957
probability of observing that point if it came from cluster 0 =  1.7551169087125603e-16
probability of observing that point if it came from cluster 1 =  0.10873310208646987
point =  8.786395077531184
probability of observing that point if it came from cluster 0 =  1.6547469448005685e-11
probability of observing that point if it came from cluster 1 =  0.6359339257676454
poi

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [21]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 4.794805314319554,  mean_2 = 8.533642579657043
var_1 = 0.5706198881423987,  var_2 = 0.5684978466220023


The values are similar to the ones obtained from K means.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [22]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s1_x.append(prob_s1_x_i)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()

4.922928569622708
Probability of coming from S_0 = 0.9999999988062223
Probability of coming from S_1 = 1.1937777501558037e-09

5.929164230830251
Probability of coming from S_0 = 0.9998663357910818
Probability of coming from S_1 = 0.00013366420891825599

3.6263679375341975
Probability of coming from S_0 = 0.9999999999999996
Probability of coming from S_1 = 3.598081525740268e-16

9.631291726546957
Probability of coming from S_0 = 2.4216682052105593e-15
Probability of coming from S_1 = 0.9999999999999976

8.786395077531184
Probability of coming from S_0 = 3.906643895071176e-11
Probability of coming from S_1 = 0.9999999999609336

5.15598286007786
Probability of coming from S_0 = 0.9999999823292052
Probability of coming from S_1 = 1.7670794792082082e-08

7.642846394028064
Probability of coming from S_0 = 1.9867825474058176e-05
Probability of coming from S_1 = 0.999980132174526

4.056745510565915
Probability of coming from S_0 = 0.9999999999999472
Probability of coming from S_1 = 5.277744061

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [23]:
hard_assignments = []
for p_s0, p_s1 in zip(prob_s0_x, prob_s1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0) 
    else:
        hard_assignments.append(1) 

for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")

Data point 0 belongs to Cluster 0
Data point 1 belongs to Cluster 0
Data point 2 belongs to Cluster 0
Data point 3 belongs to Cluster 1
Data point 4 belongs to Cluster 1
Data point 5 belongs to Cluster 0
Data point 6 belongs to Cluster 1
Data point 7 belongs to Cluster 0
Data point 8 belongs to Cluster 0
Data point 9 belongs to Cluster 1
